In [25]:
!pip install nltk
!pip install gensim

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=c1d9188a78187da4da162e1b7bb059aa46a3e11b74b6be5bf761716d98fc67df
  Stored in directory: c:\users\milan\appdata\local\pip\cache\wheels\d6\b7\9c\9880636bd2f16fcca95037d4f4734dd0fcf1338d764e69d43c
Successfully built smart-open


DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [46]:

from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding, GRU, Bidirectional
from keras.models import Model 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from nltk.tokenize import RegexpTokenizer
import tensorflow as tf
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale
from keras.models import Sequential, Model 

In [ ]:
def ensemble_CNN_BiGRU(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1D CNN
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel BiGRU
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [10]:
data = pd.read_csv('dataset/train_preprocessed.csv')
data.dropna(inplace = True)

In [21]:
train_stratify_y = data['class']
train_x, test_x, train_y, test_y = train_test_split(data['text'],train_stratify_y,test_size=0.2,random_state=42,stratify=train_stratify_y)
train_x.shape,train_y.shape,test_x.shape,test_y.shape

((15485,), (15485,), (3872,), (3872,))

In [22]:
tokenizer = RegexpTokenizer(r'\w+')
train_x = train_x.apply(tokenizer.tokenize)
test_x = test_x.apply(tokenizer.tokenize)

In [31]:
import multiprocessing
from time import time 
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

#BUILD_VOCAB()
t = time()
w2v_model.build_vocab(train_x, progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#TRAIN()
w2v_model.train(train_x, total_examples=w2v_model.corpus_count, epochs=1000, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.0 mins
Time to train the model: 2.44 mins


In [38]:
train_x, test_x, train_y, test_y = train_test_split(data['text'],train_stratify_y,test_size=0.2,random_state=42,stratify=train_stratify_y)
train_x.shape,train_y.shape,test_x.shape,test_y.shape

((15485,), (15485,), (3872,), (3872,))

In [40]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), lowercase=True, max_features=11000)
matrix = vectorizer.fit_transform(np.array(train_x))
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

vocab size : 11000


In [41]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [44]:

train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, train_x)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, test_x)])
test_vecs_w2v = scale(test_vecs_w2v)

print ('shape for training set : ',train_vecs_w2v.shape,
      '\nshape for test set : ', test_vecs_w2v.shape)

d:\projects\python projects\venv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
d:\projects\python projects\venv\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


shape for training set :  (15485, 300) 
shape for test set :  (3872, 300)


In [47]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=300))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               38528     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [48]:
history = model.fit(train_vecs_w2v, train_y, epochs=20, batch_size=50,
                   validation_data=(test_vecs_w2v, test_y))
loss, accuracy = model.evaluate(train_vecs_w2v, train_y, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_vecs_w2v, test_y, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 15485 samples, validate on 3872 samples
Epoch 1/20
15485/15485 [==============================] - 1s 73us/step - loss: 0.0388 - accuracy: 0.0229 - val_loss: -0.6652 - val_accuracy: 0.0227
Epoch 2/20
15485/15485 [==============================] - 0s 27us/step - loss: -1.4286 - accuracy: 0.0226 - val_loss: -2.2192 - val_accuracy: 0.0227
Epoch 3/20
15485/15485 [==============================] - 0s 28us/step - loss: -3.0399 - accuracy: 0.0226 - val_loss: -3.8818 - val_accuracy: 0.0227
Epoch 4/20
15485/15485 [==============================] - 0s 27us/step - loss: -4.7528 - accuracy: 0.0226 - val_loss: -5.6453 - val_accuracy: 0.0227
Epoch 5/20
15485/15485 [==============================] - 0s 26us/step - loss: -6.5675 - accuracy: 0.0226 - val_loss: -7.5099 - val_accuracy: 0.0227
Epoch 6/20
15485/15485 [==============================] - 0s 27us/step - loss: -8.4852 - accuracy: 0.0226 - val_loss: -9.4812 - val_accuracy: 0.0227
Epoch 7/20
15485/15485 [==============================] - 